In [1]:
import itertools
import numpy as np
import pandas as pd

In [2]:
big_list = [1,3,4,5,6,7,8]

In [3]:
def digital_rooter(l, doors=None):
    
    # a rather inelegant way to find the digital root
    def _dr(_l):
        s = [int(c) for c in str(sum(_l))]
        if len(s) > 1:
            return _dr(s)
        else:
            return s[0]
    
    # finds all n-length sublists in a list
    def _split(_l, _n, replacement=False):
        if replacement:
            return [list(i) for i in itertools.combinations_with_replacement(_l, _n)]
        return [list(i) for i in itertools.combinations(_l, _n)]

    # finds all possible ways to split in groups of 3, 4, or 5
    nums_list = _split([3,4,5], 2, replacement=True)
    nums_list = [n for n in nums_list if sum(n) <= len(l)]
    
    # returns the difference between two lists
    def _diff(_a, _b):
        a_ = _a
        b_ = _b
        if len(_b) > len(_a):
            a_ = _b
            b_ = _a
        return sorted(list(set(a_) - set(b_)))
    
    # returns all possible sublists from dividing a list
    def _combo_cut(_l, _n):
        _len = len(_l)
        _lofl = []
        if _n > _len:
            print('error!')
            return None
        _lofl.append(_split(_l, _n))
        _lofl.append([_diff(_l, _lofl[0][i]) for i in range(len(_lofl[0]))]) 
        return [list(z) for z in zip(*_lofl)]
    
    # iterates for multiple combo cuts
    def _list_splitter(_l, _nums):
        for i, num in enumerate(_nums[:-1]):
            tier1 = _combo_cut(l, num)
            _tier1 = []
            if sum(_nums) < len(_l):
                _tier2 = []
                for p in tier1:
                    tier2 = [[p[0]] + [*t] for t in _combo_cut(p[1], _nums[i+1])]
                    _tier2 += [*tier2]
                _tier1 += [*_tier2]
                return _tier1
            else:
                return tier1
    
    drs = []
    ls = []
    for nums in nums_list:
        ls += [*_list_splitter(l, nums)]
    for s in ls:
        drs.append([_dr(s_) for s_ in s])
    
    # checks if list has duplicates
    def _dr_check(_l):
        if abs(len(_l) - len(set(_l))) > 0:
            return False
        else:
            return True

    # checks if list is a subset of target    
    def _dr_check_2(_l, _t):
        return set(_l).issubset(set(_t))
    
    mask = [_dr_check(dr) for dr in drs]
    ls = np.array(ls)[mask]
    
    # returns dataframe of results 
    doors = sorted(doors)
    if doors is not None:
        df = pd.DataFrame(columns=list(range(10)))
        for i, j in enumerate(ls):
            out = []
            for k in j:
                if (len(k) in [3,4,5]) and (_dr(k) in doors):
                    df.loc[i, _dr(k)] = str(k)
                else:
                    out += [*k]
            if len(out) > 0:
                df.loc[i, 0] = str(sorted(out))
            else:
                df.loc[i, 0] = ''
        df[10] = df[0].str.len()
        # return df.sort_values(by=10, ascending=True)
        df = df[df[10]==df[10].min()]
        cols = [0]+doors
        df = df[cols].dropna(
            subset=doors,                 # drop rows if doors cols are nan
            how='all'
        ).drop_duplicates().sort_values(
            by=cols[:-1]
        ).dropna(
            axis=1,                         # drop columns if they are nan
            how='all'
        ).reset_index(drop=True)
        return df

In [5]:
digital_rooter([1,3,4,5,6,7,8], doors=[3,7,8])

,0,3,7,8
0,[1],NaN,"[3, 5, 8]","[4, 6, 7]"
1,[1],NaN,"[3, 6, 7]","[4, 5, 8]"
2,[1],NaN,"[4, 5, 7]","[3, 6, 8]"
3,[5],"[1, 3, 8]",NaN,"[4, 6, 7]"
4,[5],"[1, 4, 7]",NaN,"[3, 6, 8]"
5,[5],"[6, 7, 8]",NaN,"[1, 3, 4]"
6,[6],"[1, 3, 8]","[4, 5, 7]",NaN
7,[6],"[1, 4, 7]","[3, 5, 8]",NaN
8,[6],"[3, 4, 5]","[1, 7, 8]",NaN
